In [85]:
'''
Name: Jubin Soni (jas1464)
Question#1
Machine Learning Homework#3
'''

# %load tools.py
from collections import defaultdict
import re
import math
from unidecode import unidecode

spchars = re.compile('\`|\~|\!|\@|\#|\$|\%|\^|\&|\*|\(|\)|\_|\+|\=|\\|\||\{|\[|\]|\}|\:|\;|\'|\"|\<|\,|\>|\?|\/|\.|\-')

# Utility function that does the following to the text:
# - Convert to unicode
# - Convert to lowercase
# - Remove special chars
def make_text_parsable(text):
    # convert to unicode
    text = unidecode(text) #.decode('utf-8', 'ignore'))
    # convert text to lowercase
    text = text.lower()
    # remove special characters
    text = spchars.sub(" ", text)
    return(text)

#
# Tokenize by whitespace. Use the defaultdict(int) whichsets the default 
# factory to int which makes it  the default dict useful for counting. 
#
def count_words(text, wc=None):
    if wc == None:
        wc = defaultdict(int)
    tokens = text.split(" ")
    for t in tokens:
        wc[t] += 1  
    return(wc)

#
# Main function. Opens the file and calls helper functions to parse
# Returns the sorted word count
#
def extract_info(filename):
    import json
    wc = defaultdict(int)
    df = defaultdict(set)
    count = 0
    with open(filename) as fin:
        for line in fin:
            count += 1
            current = json.loads(line)
            text = make_text_parsable(current["abstract"] + " " + \
                current["description"] + " " + current["title"])
            wc = count_words(text, wc)
    
    sorted_wc = sorted(wc.items(), key=lambda x: x[1], reverse=True)
    
    return sorted_wc

# Section A: Term frequency with unigrams

In [127]:
# Question 1: Without modifying the code, which is the most common word?
dataFile = 'data_file.txt'
print 'Output:'
print extract_info(dataFile)[0:4] #Fetching only top 5 results


#Answer1:
print "\nAnswer1: Most common word: 'the'. \nIn general, most common word is space '' but since word is asked. 'the' is most common."

Output:
[('', 1919), ('the', 608), ('and', 540), ('of', 452)]

Answer1: Most common word: 'the'. 
In general, most common word is space '' but since word is asked. 'the' is most common.


### Answer1: Most common word: 'the'. In general, most common word is space '' but since word is asked. 'the' is most common. 

In [125]:
#Question2: Notice that a lot of the more common words don’t give much info since they are what we can stopwords.
#Stop words are natural language words which have very little meaning, such as "and", "the", "a", "an", and similar words.
#Now, look at the file called stopwords and modify tools.py so that it excludes stopwords in the counting.
#What are the 3 most common words now?

from nltk.tokenize import word_tokenize
import string
import nltk

stop_words = word_tokenize(open('stopwords.txt').read())
data_file = make_text_parsable(open('data_file.txt').read())
data_file = word_tokenize(data_file)

cleaned_text = filter(lambda x: x not in stop_words, data_file)

#Answer2:
print "Answer2: Most common 3 words: 'data', 'with', 'python'.\n\nOutput:"
print (nltk.FreqDist(cleaned_text)).most_common(200)


Answer2: Most common 3 words: 'data', 'with', 'python'.

Output:
[('data', 231), ('with', 133), ('python', 122), ('time', 113), ('abstract', 83), ('description', 81), ('title', 79), ('learning', 77), ('talk', 75), ('analysis', 61), ('ll', 52), ('more', 51), ('use', 48), ('3', 43), ('pandas', 42), ('machine', 39), ('10', 38), ('models', 37), ('saturday', 36), ('tools', 36), ('analytics', 33), ('code', 33), ('science', 31), ('discuss', 29), ('based', 29), ('spark', 28), ('model', 28), ('new', 27), ('sunday', 27), ('20', 24), ('tutorial', 23), ('learn', 23), ('both', 23), ('work', 21), ('techniques', 21), ('50', 21), ('5', 21), ('web', 21), ('methods', 20), ('1', 20), ('library', 20), ('2', 19), ('performance', 19), ('world', 19), ('coming', 19), ('algorithm', 19), ('software', 19), ('algorithms', 18), ('numpy', 18), ('building', 18), ('soon', 18), ('30', 18), ('40', 18), ('series', 17), ('applications', 17), ('statistical', 17), ('space', 17), ('different', 17), ('4', 17), ('deep', 17), 

### Answer2: Most common 3 words: 'data', 'with', 'python'.

In [135]:
#Question3: Given that this is a list of talks about data and python, do you think that there are some words
#that don’t add much value? Name 3 of them and add them to the stopwords list

#Answer3: If we look at 3 most common words we find that there are words like 'with', 'data', 'python' that do not add much value.
#However if we look at 50 most common words we should notice more trivial words: 'with', 'll', '3' that occur more frequenctly. Hence should be removed.

stop_words.append('with')
stop_words.append("data")
stop_words.append("python")
stop_words.append("abstract")#json object
stop_words.append("title")#json object
stop_words.append("description")#json object
stop_words.append("time")#json object

print "\nAnswer3: Words that dont add value: 'with', 'data', 'python'."


Answer3: Words that dont add value: 'with', 'data', 'python'.


### Answer3: Words that dont add value: 'with', 'data', 'python'.

In [97]:
#Question4: Now run your code again with the new list of stopwords.  What’s the most common word now?
cleaned_text = filter(lambda x: x not in stop_words, data_file)
print 'Output:'
print (nltk.FreqDist(cleaned_text)).most_common(10)

print "\nAnswer4: Most common words now are: 'learning', 'talk', 'analysis'.\n"

[('learning', 77), ('talk', 75), ('analysis', 61), ('ll', 52), ('more', 51), ('use', 48), ('3', 43), ('pandas', 42), ('machine', 39), ('10', 38)]

Answer4: Most common word now is 'learning', 'talk', 'analysis'.



### Answer4: Most common word now is 'learning', 'talk', 'analysis'. 

# Section B: Using TF-IDF

In [104]:
#Question5: Compute TF-IDF for the corpus. Note that you will need to compute a separate score for TF per document.
#Again in our file we have 79 documents

from nltk.tokenize import word_tokenize
import nltk
import math
import numpy
import json

spchars = re.compile('\`|\~|\!|\@|\#|\$|\%|\^|\&|\*|\(|\)|\_|\+|\=|\\|\||\{|\[|\]|\}|\:|\;|\'|\"|\<|\,|\>|\?|\/|\.|\-')

def make_text_parsable(text):
    # convert to unicode
    text = unidecode(text) #.decode('utf-8', 'ignore'))
    # convert text to lowercase
    text = text.lower()
    # remove special characters
    text = spchars.sub(" ", text)
    return(text)

stopWordsFile = 'data_file.txt'
dataFile = 'data_file.txt'
stop_words = word_tokenize(make_text_parsable(open(stopWordsFile).read()))
#data_file = word_tokenize(make_text_parsable(open(dataFile).read()))
#print data_file
#print stop_words

def TF(word, data_file):
    tf = []
    for document in data_file:
        current = json.loads(document)
        document = make_text_parsable(current["abstract"] + " " + \
                current["description"] + " " + current["title"])
        if word in document:
            term = document.count(word)
            term_count = float(len(nltk.word_tokenize(make_text_parsable(document))))
            tf.append(term / term_count)
        else:
            tf.append(0)
    return tf

def IDF(word, data_file):
    total_documents = len(data_file)
    if word in data_file:
        word_count = float(data_file.count(word))
        return math.log10(total_documents/word_count)

def TFIDF(word):
    Idf = IDF(word, open(dataFile).read())
    Tf = TF(word, open(dataFile).readlines())
    if Idf is None:
        Idf = 0
    TFIDF = numpy.array(Tf) * Idf
    return max(TFIDF)

d = {}
for word in stop_words:
    d[word] = TFIDF(word)

print 'Output: '
print d #prints TF-IDF score for datafile

{'limited': 0.023034980902426019, 'similarity': 0.029973228162192896, 'monte': 0.0, 'subsetting': 0.016530085963202726, 'bytecode': 0.032308804382623513, 'consists': 0.010174960889415175, 'appetite': 0.010586289095583023, 'relationships': 0.012018250905613575, 'looking': 0.027641977082911225, 'swam': 0.044947364223654224, 'aggregations': 0.015745429983936774, 'presents': 0.034350087313959846, 'bike': 0.085753449859718264, 'under': 0.060230467315107117, 'teaching': 0.029352321366416495, 'worth': 0.029352321366416495, 'risk': 0.021641323515092771, 'internet': 0.016313297950570557, 'practicality': 0.020993906645249027, 'every': 0.12516538893665949, 'updates': 0.025385489157775613, 'reforms': 0.020905696953462269, 'affect': 0.030330396245304973, 'bringing': 0.032308804382623513, 'tagging': 0.048947914966073709, 'pymc': 0.0, 'four': 0.095035241568622267, 'school': 0.15899259941440821, 'basics': 0.044781003880502636, 'cause': 0.054663971650094983, 'companies': 0.10475439242146246, 'solution'

In [130]:
#Question6: Rank each unique word by their TF-IDF score. Which one has the highest score? Use only the highest for each word.
#So if a word appears twice only use the highest score

string = ''
for w in sorted(d, key=d.get, reverse=True):
    string = string + w + " " + str(d[w]) + " \n"

print "Answer6: Words: 'break', 'fast', 'talks' have the highest scores.\n"
print "Output:"
print string

Answer6: Words: 'break', 'fast', 'talks' have the highest scores.

Output:
break 4.4984346202 
fast 3.72028336982 
as 2.35750777821 
a 2.33387967899 
talks 2.2492173101 
k 2.22273944374 
light 2.06522891745 
re 2.0492134283 
b 1.96356276026 
soon 1.8698103517 
coming 1.79937384808 
g 1.79801937499 
o 1.73359717774 
u 1.67365815773 
c 1.56697674952 
talk 1.56522891745 
l 1.44165677413 
r 1.34451817839 
s 1.28095067599 
n 1.24980754193 
min 1.2356260599 
i 1.21145449845 
ratio 1.18139737026 
t 1.14883335476 
so 1.09702390581 
com 1.09601097121 
eg 1.06823462109 
e 1.06712447596 
ing 1.05820878881 
co 0.948964600922 
m 0.925750941808 
in 0.898721311305 
key 0.855317174118 
on 0.772720299182 
io 0.727488091582 
is 0.727255163411 
and 0.724488872347 
not 0.688815391576 
at 0.644458093469 
an 0.64133916147 
no 0.569690215312 
mining 0.552839541658 
d 0.510555776049 
matplotlib 0.499826068912 
state 0.499826068912 
us 0.483129849978 
p 0.437700258574 
university 0.424956898115 
plot 0.4221627

### Answer6: Words: 'break', 'fast', 'talks' have the highest scores. After ranking each unique word with their TF-IDF scores. 

In [132]:
#Question7: Is the space ‘ ‘ still there? As one of the words. If so how can you remove it?

#Answer7: Space '' can be removed by using regex or NLTK tokenizer (better way). For my code it is not there because I used NLTK
#word_tokenizer to extract just words.

print "Efficient ways to remove space ‘ ‘ are by using Regex or NLTK word tokenizer. For my code I have removed the space using NLTK word_tokenizer, and have extracted just words."

Efficient ways to remove space ‘ ‘ are by using Regex or NLTK word tokenizer. For my code I have removed the space using NLTK word_tokenizer, and have extracted just words.


### Answer7: Efficient ways to remove space ‘ ‘ are by using Regex or NLTK word tokenizer. For my code I have removed the space using NLTK word_tokenizer, and have extracted just words."

In [134]:
#Question8: Paste the output for both Section A and Section B

print "SectionA Output: \n", (nltk.FreqDist(cleaned_text)).most_common(3)
print "\nSectionB Output: \n", string

SectionA Output: 
[('data', 231), ('with', 133), ('python', 122)]

SectionB Output: 
break 4.4984346202 
fast 3.72028336982 
as 2.35750777821 
a 2.33387967899 
talks 2.2492173101 
k 2.22273944374 
light 2.06522891745 
re 2.0492134283 
b 1.96356276026 
soon 1.8698103517 
coming 1.79937384808 
g 1.79801937499 
o 1.73359717774 
u 1.67365815773 
c 1.56697674952 
talk 1.56522891745 
l 1.44165677413 
r 1.34451817839 
s 1.28095067599 
n 1.24980754193 
min 1.2356260599 
i 1.21145449845 
ratio 1.18139737026 
t 1.14883335476 
so 1.09702390581 
com 1.09601097121 
eg 1.06823462109 
e 1.06712447596 
ing 1.05820878881 
co 0.948964600922 
m 0.925750941808 
in 0.898721311305 
key 0.855317174118 
on 0.772720299182 
io 0.727488091582 
is 0.727255163411 
and 0.724488872347 
not 0.688815391576 
at 0.644458093469 
an 0.64133916147 
no 0.569690215312 
mining 0.552839541658 
d 0.510555776049 
matplotlib 0.499826068912 
state 0.499826068912 
us 0.483129849978 
p 0.437700258574 
university 0.424956898115 
plot